# BASICS 02 - A layer of HOTS in PyTorch

In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import tonic, torch
from utils import timesurface, get_loader
import matplotlib.pyplot as plt

In [11]:
transform = tonic.transforms.NumpyAsType(int)
dataset = tonic.datasets.POKERDVS(save_to='../../Data/', train=True,transform=transform)
loader = get_loader(dataset, shuffle=True)

In [12]:
events, target = next(iter(loader))

In [114]:
class hotslayer(torch.nn.Module):
    def __init__(self, ts_size, n_neurons, bias=False, homeostasis = True):
        super(hotslayer, self).__init__()
        self.synapses = torch.nn.Linear(ts_size, n_neurons, bias=bias)
        torch.nn.init.uniform_(self.synapses.weight, a=0, b=1)
        self.cumhisto = torch.ones([n_neurons])
        self.learning_flag = True
        self.homeo_flag = homeostasis
        
    def homeo_gain(self):
        lambda_homeo = .25
        gain = torch.exp(lambda_homeo*(self.cumhisto.size(dim=0)*self.cumhisto/self.cumhisto.sum()-1))
        return gain

    def forward(self, ts):
        beta = self.synapses(ts.ravel())/torch.linalg.norm(self.synapses.weight.data, dim=1)
        if self.homeo_flag:
            beta = self.homeo_gain()*beta
        n_star = torch.argmax(beta)
        #if learning_flag:
            
        return n_star
    
    #def learning_rule(self, ts):

In [115]:
n_neurons = 4
all_ts = timesurface(events.squeeze(), dataset.sensor_size, dataset.ordering)
ts_size = torch.tensor(all_ts[0].shape).prod()
layer = hotslayer(ts_size, n_neurons)
n_star = layer(all_ts[10])

tensor([1., 1., 1., 1.]) tensor([1.8937, 1.8110, 1.8568, 1.8534], grad_fn=<DivBackward0>)


In [87]:
layer.cumhisto.size(dim=0)

4